# Fine-tuning Transformers model for correct first page prediction

This notebook covers one of the approaches to training a model for predicting whether a page of the document is the first one or not -- a feature that would allow correct splitting for PDFs that consist of more than one actual document (we assume that the pages are already sorted). The approach used is fine-tuning Transformers model (BERT) with our document-related dataset.

Before you start, make sure you have **installed** and **initialized** the konfuzio_sdk package as shown in the readme of the [repository](https://github.com/konfuzio-ai/Python-SDK).

In [ ]:
!pip install konfuzio-sdk

In [ ]:
!konfuzio_sdk init

Also, you will need to install the Transformers-related packages:

In [ ]:
!pip install transformers datasets

Importing necessary libraries and packages:

In [1]:
import os
import torch

import numpy as np
import pandas as pd

from datasets import load_dataset, load_metric
from nltk import word_tokenize
from konfuzio_sdk.data import Project
from tqdm import tqdm
from transformers import BertTokenizer, AutoModelForSequenceClassification, AutoConfig, \
                        TrainingArguments, DataCollatorWithPadding, Trainer

Setting seed for reproducibility purposes:

In [2]:
seed_value = 42
os.environ['PYTHONHASHSEED'] = str(seed_value)

Initializing the config file, model and the tokenizer:

In [10]:
configuration = AutoConfig.from_pretrained('bert-base-uncased')
configuration.num_labels = 2

In [11]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', config=configuration)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, max_length=10000, 
                                          padding="max_length", truncate=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Loading the project 1644's documents and creating a .csv dataset for usage with Transformers-native Dataset class:

In [3]:
my_project = Project(id_=1644)
train_data = my_project.documents
test_data = my_project.test_documents

In [4]:
train_data_texts = []
train_data_labels = []

for doc in tqdm(train_data):
    for page in doc.pages():
        train_data_texts.append(page.text)
        if page.number == 1:
            train_data_labels.append(1)
        elif page.number != 1 and int(page.number):
            train_data_labels.append(0)
        else:
            print(page.number)

100%|██████████████████████████████████████| 1443/1443 [00:02<00:00, 614.22it/s]


In [5]:
test_data_texts = []
test_data_labels = []

for doc in tqdm(test_data):
    for page in doc.pages():
        test_data_texts.append(page.text)
        if page.number == 1:
            test_data_labels.append(1)
        elif page.number != 1 and int(page.number):
            test_data_labels.append(0)
        else:
            print(page.number)

100%|████████████████████████████████████████| 286/286 [00:00<00:00, 629.91it/s]


In [13]:
tokenized_train = tokenizer(train_data_texts)

In [25]:
train_df = pd.DataFrame({'text': train_data_texts, 'label': train_data_labels})
train_df.to_csv('train_1644.csv')

In [15]:
tokenized_test = tokenizer(test_data_texts)

In [26]:
test_df = pd.DataFrame({'text': test_data_texts, 'label': test_data_labels})
test_df.to_csv('test_1644.csv')

In [27]:
tokenized = load_dataset('csv',
                      data_files={'train': 'train_1644.csv',
                                 'test': 'test_1644.csv'})

2022-09-23 16:27:58,479 [datasets.builder    ] [MainThread] [WARNING ] [_create_builder_conf][0463] Using custom data configuration default-0ae9c0752663b210


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /Users/macbookpro/.cache/huggingface/datasets/csv/default-0ae9c0752663b210/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Setting the training arguments:


In [28]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2634
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 435
    })
})

In [20]:
arguments = TrainingArguments(
    do_predict=True,
    output_dir='model', 
    evaluation_strategy="steps", 
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    num_train_epochs=25,
    logging_steps=1000, 
    logging_strategy='steps', 
    save_strategy='no',
    save_total_limit=2,
    seed=42,

)

data_collator = DataCollatorWithPadding(tokenizer, return_tensors="pt")

Tokenizing our dataset:

In [29]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [30]:
tokenized = dataset.map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Defining our metric of choice which is accuracy:

In [21]:
metric = load_metric('accuracy')

In [22]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

Initializing the Trainer class and starting the training process:

In [23]:
trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [24]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Unnamed: 0, text. If Unnamed: 0, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/anaconda3/envs/konfuzio/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2634
  Num Epochs = 25
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 16475


ValueError: type of [101, 2002, 2140, 19510, 2401, 17076, 16515, 20058, 2078, 1010, 19181, 2078, 4315, 2310, 2869, 17322, 14544, 1045, 7295, 27969, 11039, 5110, 8865, 2497, 27665, 2099, 10424, 2923, 3854, 1016, 24185, 8661, 3854, 16417, 24930, 4078, 14405, 24449, 2015, 2019, 17183, 14405, 29181, 13473, 10820, 16216, 6914, 21436, 8040, 26378, 6199, 18337, 2583, 7295, 2102, 27215, 16417, 11113, 19845, 8661, 6155, 8040, 26378, 6199, 27412, 2015, 17076, 15878, 4140, 4895, 3334, 13578, 4221, 2102, 1012, 1045, 8093, 2310, 2869, 17322, 15532, 5620, 6299, 22930, 25091, 2022, 8609, 9033, 2063, 1999, 29536, 25301, 20800, 27843, 6914, 1012, 4895, 8043, 2063, 14405, 29181, 13102, 6820, 7512, 2022, 13777, 6528, 1045, 28362, 15544, 5332, 3683, 27843, 6914, 1012, 2978, 2618, 14068, 2102, 12155, 19731, 2078, 9033, 2063, 4830, 5886, 3280, 25312, 6914, 10047, 14405, 29181, 5285, 4877, 5794, 4305, 2290, 6151, 4138, 3775, 2290, 1012, 16216, 10609, 9033, 2063, 8740, 2818, 14017, 5403, 8529, 21515, 2063, 20

In [ ]:
trainer.save_model()

## Metrics & prediction

Evaluating the trained model's performance:

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('model', 
                                                           config=configuration)
tokenizer = BertTokenizer.from_pretrained('model', do_lower_case=True, 
                                          max_length=10000, padding="max_length", truncate=True)

We calculate our custom metric via the following function that determines how many ground-truth first pages were actually predicted as first pages. The logic behind this approach suggests that by determining first pages correctly we can consecutively split documents correctly, using each first page as a separator (since it means a start of a new document).

In [ ]:
def calculate_metrics(texts, labels):
    true_positive = 0
    false_positive = 0
    false_negative = 0
    true_negative = 0
    
    for i, test in tqdm(zip(labels, texts)):
        inputs = tokenizer(test, truncation=True, return_tensors="pt")
        with torch.no_grad():
            logits = model(**inputs).logits
        pred = logits.argmax().item()
        
        if i == 1 and pred == 1:
            true_positive += 1
        elif i == 1 and pred == 0:
            false_negative += 1
        elif i == 0 and pred == 1:
            false_positive += 1
        elif i == 0 and pred == 0:
            true_negative += 1

        
    
    if true_positive + false_positive != 0:
        precision = true_positive / (true_positive + false_positive)
    else:
        precision = 0
    
    if true_positive + false_negative != 0:
        recall = true_positive / (true_positive + false_negative)
    else:
        recall = 0
    
    if precision + recall != 0:
    
        f1 = 2 * precision * recall / (precision + recall)
    
    else:
        
        f1 = 0
    
    acc = (true_positive + true_negative) / len(texts)
    
    return precision, recall, f1, acc

In [10]:
precision, recall, f1, acc = calculate_metrics(pages_test_docs, pages_labels_test)

435it [13:05,  1.81s/it]


In [12]:
print('\n Precision: {} \n Recall: {} \n F1-score: {}'.format(precision, recall, f1))


 Precision: 0.7225433526011561 
 Recall: 0.8741258741258742 
 F1-score: 0.7911392405063291
